# Hubro Platform - AI integration examples

This notebook demonstrates multiple approaches of integrating LLM or classic ML algorithms into your Hubro study.
Preferably, run this notebook in a prepared environment as documented in http://docs.hubroplatform.no/ai-inference.html#preparing-your-environment

Before continuing, make sure that your MLFlow server is running locally on port 5000
 
```mlflow server```

Run the following cell to install dependencies and set up API keys to the LLM engine of your choice.

In [1]:
! pip install openai langchain langchain_community openai langchain_openai replicate scikit-learn pandas
import os
os.environ['OPENAI_API_KEY']="YOUR OPENAI API KEY"
os.environ['REPLICATE_API_TOKEN']="YOUR REPLICATE API TOKEN"

The example below demonstrates use of a prompt template-based LLMChain using OpenAI's ChatGPT. 

In [22]:
#source: https://mlflow.org/docs/latest/python_api/mlflow.langchain.html
from langchain_community.llms.openai import OpenAI
import mlflow
from langchain import LLMChain, PromptTemplate
from langchain.llms import Mlflow

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("LLMChain using OpenAI")

assert "OPENAI_API_TOKEN" in os.environ, "Please set the OPENAI_API_TOKEN environment variable."

llm = OpenAI()

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(
        input_variables=["adjective"],
        template="Tell me a {adjective} joke",
    ),
)
with mlflow.start_run():
    mlflow.langchain.log_model(llm_chain, "openai_joke_generator")

2024/09/06 12:01:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run masked-turtle-391 at: http://127.0.0.1:5000/#/experiments/965190641623225329/runs/0542ecbf8b734a938c5d6197d992e7a6.
2024/09/06 12:01:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/965190641623225329.


Model should be tracked by now. You can view the run by visiting MLFlow's UI http://127.0.0.1:5000 in the browser. The model is also now ready to be deployed to Hubro Platform. As explained in https://docs.hubroplatform.no/ai-inference.html#deploying-models-to-hubro and documented later in this notebook, the basic two options for model deployment are using KServe or KNative.

Another example below demonstrates creation of a simple answering using LCEL definition using another LLM provider (Replicate). This example also makes use of model autologging feature of MLFlow. 

In [9]:
#https://mlflow.org/docs/latest/llms/langchain/autologging.html#example-code-of-langchain-autologging
import os
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_community.llms import Replicate
import mlflow  
import replicate

assert "REPLICATE_API_TOKEN" in os.environ, "Please set the REPLICATE_API_TOKEN environment variable."

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("LCEL using Replicate")

# Enable mlflow langchain autologging
mlflow.langchain.autolog(
    log_input_examples=True,
    log_model_signatures=False,
    log_models=True,
    log_inputs_outputs=True,
    registered_model_name="replicate_answering_model",
)

prompt_with_history_str = """
Please answer this question: {question}
"""
prompt_with_history = PromptTemplate(
    input_variables=["query"], template=prompt_with_history_str
)

def extract_question(input):
    return input

llm = Replicate(
    model="meta/meta-llama-3-70b-instruct",
    model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1},
)

chain_with_history = (
        {
            "question": itemgetter("query") | RunnableLambda(extract_question),
        }
        | prompt_with_history
        | llm
        | StrOutputParser()
)

inputs = {"query": "What is the highest mountain in the world?"}

print(chain_with_history.invoke(inputs))

# Verification can be done using

model_name = "replicate_answering_model"
model_version = 1
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")
print(loaded_model.predict(inputs))

2024/09/08 08:23:54 WARNING mlflow.utils.autologging_utils.config: The log_inputs_outputs option is deprecated and will be removed in a future release. Please use the log_traces option in `mlflow.<flavor>.autolog` to log traces (including inputs and outputs) of the model.
2024/09/08 08:24:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/homebrew/anaconda3/envs/mlflow-env/lib/python3.10/site-packages/mlflow/langchain/_langchain_autolog.py:363: UserWarning: MLflow autologging does not support logging models containing BaseRetriever because logging the model requires `loader_fn` and `persist_dir`. Please log the model manually using `mlflow.langchain.log_model(model, artifact_path, loader_fn=..., persist_dir=...)`"
2024/09/08 08:24:06 INFO mlflow.langchain._langchain_autolog: Signature is automatically generated for logged model if input_example is provided. To disable log_model_signatures, please also disable log_input_examples.
2024/09/08 08:24:

2024/09/08 08:24:13 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": {
    "query": "How can I conmfort myself during my period?"
  }
}. Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model uri and serving input example. A serving input example can be generated from model input example using `mlflow.models.convert_input_example_to_serving_input` function.
Got error: 1 tasks failed. Errors: {0: 'error: ReplicateError(type=None, title=\'Free time limit reached\', status=402, detail=\'You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.\', instance=None) Traceback (most recent call last):\n  File "/opt/homebrew/anaconda3/envs/mlflow-env/lib/python3.10/site-packages/mlflow/langchain/api_request_paral



I'm happy to help!

Dealing with periods can be uncomfortable, both physically and emotionally. Here are some tips to help you comfort yourself during your period:

1. **Warmth therapy**: Apply a warm heating pad or a hot water bottle to your lower abdomen to ease cramps. You can also take a warm bath or shower to relax your muscles.
2. **Cozy up with comfort foods**: Reach for your favorite comfort foods like chocolate, soup, or mac and cheese. These can help lift your mood and provide a sense of comfort.
3. **Stay hydrated**: Drink plenty of water to help flush out toxins and reduce bloating. Herbal teas like chamomile or peppermint can also be soothing.
4. **Get comfortable clothes**: Wear loose, comfortable clothing that doesn't irritate your skin. You might find that soft, stretchy fabrics like cotton or modal feel nice against your skin.
5. **Pamper yourself**: Treat yourself to a relaxing activity, like reading a book, listening to calming music, or practicing gentle stretches

Another example presents an integration of a classic ML classifier.

In [2]:
#source: https://mlflow.org/docs/latest/python_api/mlflow.sklearn.html

import mlflow
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("Random Forest")

db = load_diabetes()

X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
with mlflow.start_run():
    rf.fit(X_train, y_train)
    mlflow.sklearn.log_model(rf)

2024/06/18 21:27:10 INFO mlflow.tracking.fluent: Experiment with name 'Random Forest' does not exist. Creating a new experiment.


TypeError: log_model() missing 1 required positional argument: 'artifact_path'

And yet another example of using Python function as a predictor.

In [3]:
#source: https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html
import mlflow
import pandas as pd

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("Function-based model")

# Define a simple function to log
def predict(model_input):
    return model_input.apply(lambda x: x * 2)

# Save the function as a model
with mlflow.start_run():
    mlflow.pyfunc.log_model("function_model", python_model=predict, pip_requirements=["pandas"])
    run_id = mlflow.active_run().info.run_id

# mlflow.pyfunc.save(model="model", python_model=predict)

model = mlflow.pyfunc.load_model(f"runs:/{run_id}/model")
data = pd.Series([1, 2, 3])

prediction = model.predict(data)
print(prediction)

2024/06/18 21:27:20 INFO mlflow.tracking.fluent: Experiment with name 'Function-based model 2' does not exist. Creating a new experiment.


AttributeError: module 'mlflow.pyfunc' has no attribute 'save'

Any of created models can be now served locally for testing using the MLServer. This is a good option for testing the behaviour and performance before putting the model into production. 

In [1]:
!mlflow models serve -m runs:/ff5d0330792c4bf7b62e2609bbc4410c/model -p 1234 --enable-mlserver

Traceback (most recent call last):
  File "/Users/muznymir/Library/Caches/pypoetry/virtualenvs/mole-dxJyzUfb-py3.11/bin/mlflow", line 8, in <module>
    sys.exit(cli())
             ^^^^^
  File "/Users/muznymir/Library/Caches/pypoetry/virtualenvs/mole-dxJyzUfb-py3.11/lib/python3.11/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/muznymir/Library/Caches/pypoetry/virtualenvs/mole-dxJyzUfb-py3.11/lib/python3.11/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/Users/muznymir/Library/Caches/pypoetry/virtualenvs/mole-dxJyzUfb-py3.11/lib/python3.11/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/muznymir/Library/Caches/pypoetry/virtualenvs/mole-dxJyzUfb-py3.11/lib/python3.11/site-packages/click/core.py"

Inference REST API endpoint can be called now by

In [ ]:
!curl -X POST -d '{"inputs":[{"query": "utterly stupid"}]}' http://localhost:1234/invocations

### Deployment to Hubro

Deployment to Hubro

#### KNative

KNative makes use of prepared docker images.  

In [ ]:
!docker login -u USERNAME -p PASSWORD 

In [ ]:
!mlflow deployments create --name replicate_answering_model -t knative:/ --model-uri wasbs://models@hubro.blob.core.windows.net/ --config image_repository=docker.io/mmuzny/replicate-test-final --config image_tag=latest --config service_template=service_template.yaml